## Reference: https://chriskang028.medium.com/statistic-hypothesis-testing-f766c129d632

In [96]:
## Import the packages
import numpy as np
from scipy import stats
from scipy.stats import norm
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp

In [113]:
import pandas as pd
import csv
import pathlib
import warnings
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, LeaveOneOut
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from scipy.io import loadmat
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import os
warnings.simplefilter("ignore")

############################################
data_type   = 'mat'
#              mat: MAT file (MATLAB)
#              csv: CSV file
mat_name    = '/Users/chiuhaoyu/Desktop/Shikoku_project/Output/WS-ES_WS-c_events_Ecomp_3stations_feature.mat'
mat_name1   = '/Users/chiuhaoyu/Desktop/Shikoku_project/Output/WS-ES_WS-c_events_Ecomp_3stations_feature.mat'

# MAT name, for data_type = 'mat' only

In [114]:
mat = loadmat(mat_name)
mat1 = loadmat(mat_name1)
Fvect = mat['C_data'][0]
Fvect1 = mat1['C_data'][0]
C1_TREMOR = Fvect[1]
C2_TREMOR = Fvect1[2]
print(f'Data: \nTa-Sa: {len(C1_TREMOR)}\nTb-Sb: {len(C2_TREMOR)}')

Data: 
Ta-Sa: 20631
Tb-Sb: 20631


In [61]:
##################################################
row_rand_C1_TREMOR = np.arange(C1_TREMOR.shape[0])
np.random.shuffle(row_rand_C1_TREMOR)
C1_TREMOR = C1_TREMOR[row_rand_C1_TREMOR[0:7500]]
##################################################
##################################################
row_rand_C2_TREMOR = np.arange(C2_TREMOR.shape[0])
np.random.shuffle(row_rand_C2_TREMOR)
C2_TREMOR = C2_TREMOR[row_rand_C2_TREMOR[0:7500]]
##################################################
print(f'Data: \nTa-Sa: {len(C1_TREMOR)}\nTb-Sb: {len(C2_TREMOR)}')

Data: 
Ta-Sa: 7500
Tb-Sb: 7500


In [101]:
N = len(C1_TREMOR)
N

7524

In [115]:
C1_TREMOR = C1_TREMOR.real
C2_TREMOR = C2_TREMOR.real

In [116]:
C1_TREMOR = C1_TREMOR[:,10]
C2_TREMOR = C2_TREMOR[:,10]

In [117]:
C1_TREMOR = C1_TREMOR / np.sqrt(np.sum(C1_TREMOR**2))
C2_TREMOR = C2_TREMOR / np.sqrt(np.sum(C2_TREMOR**2))

In [118]:
## Calculate the Standard Deviation
#Calculate the variance to get the standard deviation
#For unbiased max likelihood estimate we have to divide the var by N-1, and therefore the parameter ddof = 1
var_a = C1_TREMOR.var(ddof=1)
var_b = C2_TREMOR.var(ddof=1)

#std deviation
s = np.sqrt((var_a + var_b)/2)

## 統計量，可參考上方樣本數不同時的公式
t = (C1_TREMOR.mean() - C2_TREMOR.mean())/(s*np.sqrt(2/N))

# Compare with the critical t-value
# 自由度
df = 2*N - 2
# 計算 p-value after comparison with the t
p = 1 - stats.t.cdf(t,df=df)
# 結果
print("t = " + str(t))
print("p = " + str(2*p))
# Note that we multiply the p value by 2 because its a twp tail t-test
### You can see that after comparing the t statistic with the critical t value (computed internally) we get a good p value of 0.0005 and thus we reject the null hypothesis and thus it proves that the mean of the two distributions are different and statistically significant.
## Cross Checking with the internal scipy function
t2, p2 = stats.ttest_ind(C1_TREMOR, C2_TREMOR) 
# 結果作為交叉參照
print("t = " + str(t2))
print("p = " + str(2*p2))

t = 2.0720356660830115
p = 0.0382791100892228
t = 3.431096815666135
p = 0.0012034692767029875


In [119]:
stats.ttest_ind(C1_TREMOR, C2_TREMOR, axis=0, equal_var=True)

Ttest_indResult(statistic=3.431096815666135, pvalue=0.0006017346383514938)

In [77]:
n1 = C1_TREMOR + C2_TREMOR

In [78]:
loc, scale = norm.fit(n1)
post_treat = norm(loc=loc, scale=scale)

In [120]:
t_val, p = ttest_ind(C1_TREMOR, C2_TREMOR)

print('t = {}'.format(t_val))
print('p-value = {}'.format(p))

t = 3.431096815666135
p-value = 0.0006017346383514938


In [121]:
t_val, p = ttest_rel(C1_TREMOR, C2_TREMOR)

print('t = {}'.format(t_val))
print('p-value = {}'.format(p))

t = 5.690420081129523
p-value = 1.2844545483580378e-08


In [122]:
stats.ttest_ind(C1_TREMOR, C2_TREMOR, alternative='two-sided', trim=.2)

Ttest_indResult(statistic=0.3739654573517263, pvalue=0.7084332183308722)

In [123]:
def t_test(group1, group2):
    mean1 = np.mean(group1)
    mean2 = np.mean(group2)
    std1 = np.std(group1)
    std2 = np.std(group2)
    nobs1 = len(group1)
    nobs2 = len(group2)
    
    modified_std1 = np.sqrt(np.float32(nobs1)/
                    np.float32(nobs1-1)) * std1
    modified_std2 = np.sqrt(np.float32(nobs2)/
                    np.float32(nobs2-1)) * std2
    statistic, pvalue = stats.ttest_ind_from_stats( 
               mean1=mean1, std1=modified_std1, nobs1=nobs1,   
               mean2=mean2, std2=modified_std2, nobs2=nobs2 )
    return statistic, pvalue

In [124]:
t_test(C1_TREMOR, C2_TREMOR)

(3.4310969418218993, 0.0006017343585838898)